In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import json
import pandas as pd
import time

In [2]:
inputDF = pd.read_csv('upc_full_input.csv', encoding='latin1')

In [3]:
driver_1 = webdriver.Chrome()
driver_2 = webdriver.Chrome()

outputFileName = 'fake_upc_full_output.csv'
outputFile = open(outputFileName, 'a', encoding='utf-8')
outputFile.write('Input UPC,Walmart Listing ID,Walmart Product ID,Prodict Name,Pharmapacks,Number of Sellers,Output UPC 1,Output UPC 2\n')
outputFile.close()

In [4]:
searchHead = 'https://www.walmart.com/search/?query='
listingHead = 'https://www.walmart.com/product/'
listingTail = '/sellers'

count = 0

for input_UPC in inputDF['UPC Code']:
    input_UPC = str(input_UPC)
    searchURL = searchHead + input_UPC
    driver_1.get(searchURL)
    
    try:
        listings = driver_1.find_elements_by_class_name('product-title-link.line-clamp.line-clamp-2')
        listingIDs = [ x.get_attribute('href').split('/')[-1].split('?')[0] for x in listings ]
    except:
        outputFile = open(outputFileName, 'a', encoding='utf-8')
        outputFile.write(input_UPC +'\n')
        outputFile.close()
        continue
        
    for listingID in listingIDs:
        driver_2.get( listingHead + listingID + listingTail )
        #json_code = driver_2.find_element_by_xpath("//script[@type='application/json']").get_attribute('innerText')
        html = driver_2.page_source
        soup = BeautifulSoup(html)
        json_code = soup.find('script', id='item').get_text()
        json_parsed = json.loads(json_code)
        
        try:
            walmartProductID = json_parsed['item']['productBasicInfo']['selectedProductId']
        except:
            walmartProductID = ''
            
        try:
            output_UPC_1 = json_parsed['item']['product']['products'][walmartProductID]['upc']
        except:
            output_UPC_1 = ''
            
        try:
            output_UPC_2 = json_parsed['item']['productBasicInfo'][walmartProductID]['upc']
        except:
            output_UPC_2 = ''
        
        try:
            productDesc = json_parsed['item']['product']['products'][walmartProductID]['productAttributes']['productName'].replace(',', ';')
        except:
            productDesc = ''

        
        
        seller_card_list = soup.find_all('a', class_="seller-shipping-msg")
        sellerList = [ seller.get_text() for seller in seller_card_list ]
        involved = str( 'Pharmapacks' in sellerList )
        sellerTotal = str( len(sellerList) )
        
        
        outputFile = open(outputFileName, 'a', encoding='utf-8')
        outputFile.write(input_UPC +','+ 
                         listingID +','+ 
                         walmartProductID +','+ 
                         productDesc +','+ 
                         involved +','+ 
                         sellerTotal +','+ 
                         output_UPC_1 +','+ 
                         output_UPC_2 +'\n')
        outputFile.close()
        count += 1
    
    if count >= 100:
        #break
        pass
        